## Lab 5. Threat hunting

### Miembros
* Fernanda Esquivel - 21542

### Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/SDS-Lab-05)

In [1]:
import json
import pandas as pd
from pandas import json_normalize

In [4]:
# Configuración para mostrar más columnas en pandas
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

In [16]:
allRecords = []

try:
    with open('data/large_eve.json', 'r') as file:
        for line in file:
            line = line.strip()
            if line:  #Ignorar lineas vacías
                try:
                    record = json.loads(line)
                    allRecords.append(record)
                except json.JSONDecodeError as e:
                    print(f"Error al decodificar JSON en línea: {line[:50]}... - Error: {e}")
                    continue
except Exception as e:
    print(f"Error al procesar el archivo: {e}")

In [14]:
#Mostrar cantidad de registros total
totalRecords = len(allRecords)
print(f"Total de registros: {totalRecords}")

Total de registros: 746909


In [10]:
#Filtrar únicamente registros DNS
dnsRecords = [record for record in allRecords if record.get('event_type') == 'dns']

In [19]:
#Mostrar la nueva cantidad de registros filtrados
dnsCount = len(dnsRecords)
print(f"Total de registros DNS: {dnsCount}")

Total de registros DNS: 15749


In [21]:
if dnsCount > 0:
    print("\n4. Muestra de 2 registros DNS:")
    for i, record in enumerate(dnsRecords[:min(2, dnsCount)]):
        print(f"\nRegistro {i+1}:")
        print(json.dumps(record, indent=2))
else:
    print("\nNo hay registros DNS para normalizar.")


4. Muestra de 2 registros DNS:

Registro 1:
{
  "timestamp": "2017-07-22T17:33:16.661646-0500",
  "flow_id": 1327836194150542,
  "pcap_cnt": 22269,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 59680,
  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 15529,
    "rrname": "api.wunderground.com",
    "rrtype": "A",
    "tx_id": 0
  }
}

Registro 2:
{
  "timestamp": "2017-07-22T17:33:24.990320-0500",
  "flow_id": 2022925111925872,
  "pcap_cnt": 54352,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 38051,
  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 58278,
    "rrname": "stork79.dropbox.com",
    "rrtype": "A",
    "tx_id": 0
  }
}


In [26]:
#Normalizar la información JSON anidada y asignarla a un dataframe
if dnsCount > 0:
    df = json_normalize(dnsRecords)
    print(f"Shape del DataFrame: {df.shape}")
    
    #Examinar campos relacionados con DNS para identificar el tipo
    dnsFields = [col for col in df.columns if 'dns' in col.lower()]
    print("\nCampos relacionados con DNS:")
    for field in dnsFields:
        print(f"- {field}")
    
    #Buscar el campo que contiene el tipo de registro DNS
    dnsTypeCol = None
    
    #Posibles campos para el tipo DNS, basados en el formato observado
    possibleTypeFields = [
        'dns.type', 'dns.rrtype', 'dns.qtype', 'dns.rrname.type', 
        'dns.query.type', 'dns.answers.type', 'dns.answers.rrtype'
    ]
    
    #Comprobar cuál de estos campos existe en el DataFrame
    for field in possibleTypeFields:
        if field in df.columns:
            #Mostrar valores únicos para este campo
            unique_values = df[field].dropna().unique()
            
            #Verificar si 'A' está entre los valores
            if 'A' in unique_values or 1 in unique_values:  #El tipo A puede ser 1 en formato numérico
                dnsTypeCol = field
                break
    
    if dnsTypeCol:        
        #Filtrar por tipo A (puede ser 'A' o 1 dependiendo del formato)
        if 'A' in df[dnsTypeCol].values:
            df_type_a = df[df[dnsTypeCol] == 'A']
        elif 1 in df[dnsTypeCol].values:
            df_type_a = df[df[dnsTypeCol] == 1]
        else:
            df_type_a = pd.DataFrame()  # DataFrame vacío si no hay coincidencias
        
        print(f"\nTotal de registros DNS tipo A: {len(df_type_a)}")
        
        if not df_type_a.empty:
            print("\nPrimeras filas de registros DNS tipo A:")
            print(df_type_a.head(2))
    else:
        print("\nNo se pudo identificar automáticamente el campo que contiene el tipo de registro DNS.")
        print("Buscando en la estructura de los registros originales...")
        
        #Examinar directamente los registros para buscar indicios de tipo A
        typeARecords = []
        
        for record in dnsRecords:
            dns_field = record.get('dns', {})
            
            # Buscar 'type' = 'A' o type = 1 en diferentes ubicaciones
            if isinstance(dns_field, dict):
                # Verificar directamente en el campo dns
                if dns_field.get('type') in ['A', 1]:
                    typeARecords.append(record)
                # Verificar en respuestas (answers)
                elif 'answers' in dns_field and isinstance(dns_field['answers'], list):
                    for answer in dns_field['answers']:
                        if isinstance(answer, dict) and answer.get('type') in ['A', 1]:
                            typeARecords.append(record)
                            break
        
        if typeARecords:
            print(f"Encontrados {len(typeARecords)} registros DNS tipo A")
            print("\nEjemplo de registro DNS tipo A:")
            print(json.dumps(typeARecords[0], indent=2))
            
            # Crear un dataframe con estos registros
            df_type_a = json_normalize(typeARecords)
            print(f"Shape del DataFrame de registros tipo A: {df_type_a.shape}")
        else:
            print("No se encontraron registros DNS tipo A.")
else:
    print("\nNo hay registros DNS para normalizar o filtrar.")

Shape del DataFrame: (15749, 18)

Campos relacionados con DNS:
- dns.type
- dns.id
- dns.rrname
- dns.rrtype
- dns.tx_id
- dns.rcode
- dns.ttl
- dns.rdata

Total de registros DNS tipo A: 2849

Primeras filas de registros DNS tipo A:
                         timestamp           flow_id  pcap_cnt event_type  vlan                                   src_ip  src_port                                  dest_ip  dest_port proto dns.type  dns.id            dns.rrname dns.rrtype  dns.tx_id dns.rcode  dns.ttl dns.rdata
0  2017-07-22T17:33:16.661646-0500  1327836194150542     22269        dns   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     59680  2001:0500:0001:0000:0000:0000:803f:0235         53   UDP    query   15529  api.wunderground.com          A        0.0       NaN      NaN       NaN
1  2017-07-22T17:33:24.990320-0500  2022925111925872     54352        dns   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     38051  2001:0500:0003:0000:0000:0000:0000:0042         53   UDP    query   58278   s

In [30]:
#Filtrar dominios únicos
#Identificar la columna que contiene el nombre de dominio
domainColumn = None
possibleDomainColumns = [
    'dns.rrname', 'dns.query.rrname', 'dns.query', 'dns.answers.rrname', 
    'dns.qname', 'dns.request', 'dns.queries.rrname'
]

for col in possibleDomainColumns:
    if col in df_type_a.columns:
        print(f"Columna candidata para dominio: {col}")
        domainColumn = col
        break

if not domainColumn:
    # Buscar cualquier columna que pueda contener dominios
    for col in df_type_a.columns:
        if any('name' in col.lower() or 'domain' in col.lower() or 'host' in col.lower() for part in col.split('.')):
            print(f"Encontrada posible columna de dominio: {col}")
            domainColumn = col
            break

if domainColumn:
    
    #Crear DataFrame de dominios únicos
    uniqueDomains = df_type_a[domainColumn].dropna().unique()
    print(f"\nTotal de dominios únicos: {len(uniqueDomains)}")
    
    if len(uniqueDomains) > 0:        
        #Crear DataFrame de dominios únicos
        domainsDF = pd.DataFrame({domainColumn: uniqueDomains})
    else:
        print("No se encontraron dominios únicos.")
        domainsDF = pd.DataFrame()
else:
    print("No se pudo identificar la columna que contiene los nombres de dominio.")
    
    #Intentar buscar dominios directamente en los registros
    domainsSet = set()
    for record in typeARecords:
        dnsData = record.get('dns', {})
        
        # Buscar en diferentes campos posibles
        possibleFields = ['rrname', 'query', 'qname', 'request']
        for field in possibleFields:
            if field in dnsData and isinstance(dnsData[field], str):
                domainsSet.add(dnsData[field])
            
        # Buscar en respuestas
        if 'answers' in dnsData and isinstance(dnsData['answers'], list):
            for answer in dnsData['answers']:
                if isinstance(answer, dict) and 'rrname' in answer:
                    domainsSet.add(answer['rrname'])
    
    if domainsSet:
        print(f"Encontrados {len(domainsSet)} dominios únicos buscando directamente en los registros")
        
        uniqueDomains = list(domainsSet)
        print("\nEjemplos de dominios:")
        for domain in uniqueDomains[:min(5, len(uniqueDomains))]:
            print(f"- {domain}")
        
        # Crear DataFrame con estos dominios
        domainsDF = pd.DataFrame({'domain': uniqueDomains})
        domainColumn = 'domain'
    else:
        print("No se encontraron dominios.")
        domainsDF = pd.DataFrame()
        domainColumn = None

Columna candidata para dominio: dns.rrname

Total de dominios únicos: 177


**Prompt utilizado**: *Para el jupyter notebook adjunto, realiza una función en python que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home.*

In [31]:
#Función para obtener el TLD de un dominio
'''
Descripción:
    Obtiene el TLD (Top Level Domain) para un dominio.
    
Args:
    domain (str): El nombre de dominio completo
    
Returns:
    str: El TLD del dominio
'''
def getTLD(domain):
    if not domain or not isinstance(domain, str):
        return None
    
    # Dividir por puntos y obtener las últimas dos partes
    parts = domain.strip().split('.')
    
    if len(parts) <= 1:
        return domain  # No hay TLD para un dominio sin puntos
    elif len(parts) == 2:
        return domain  # El dominio ya es un TLD (example.com)
    else:
        # Casos como .co.uk o .com.br necesitarían tratamiento especial
        # Pero para el caso general, tomamos las últimas dos partes como TLD
        
        # Para el caso especial mencionado: safebrowsing.clients.google.com.home
        # La última parte es 'home', lo que debería ser el TLD
        if parts[-1].lower() in ['home', 'local', 'internal', 'lan', 'corp', 'intranet']:
            return parts[-1]
        
        # Para dominios como api.wunderground.com, el TLD sería wunderground.com
        return f"{parts[-2]}.{parts[-1]}"

In [32]:
#Aplicar la función para obtener el TLD y crear la columna
if not domainsDF.empty and domainColumn:
    
    # Aplicar la función a cada dominio
    domainsDF['domain_tld'] = domainsDF[domainColumn].apply(getTLD)
    
    # Eliminar todas las demás columnas, quedarse solo con domain_tld
    domainsDF = domainsDF[['domain_tld']]
    
    print(f"Shape del DataFrame final: {domainsDF.shape}")
    print("\nPrimeros 10 TLDs:")
    print(domainsDF.head(10))
    
    # Mostrar la cantidad de TLDs únicos
    unique_tlds = domainsDF['domain_tld'].nunique()
    print(f"\nTotal de TLDs únicos: {unique_tlds}")
    
    # Mostrar los TLDs más frecuentes
    print("\nTLDs más frecuentes:")
    print(domainsDF['domain_tld'].value_counts().head(10))

Shape del DataFrame final: (177, 1)

Primeros 10 TLDs:
         domain_tld
0  wunderground.com
1       dropbox.com
2         aoltw.net
3              home
4       mozilla.com
5    metasploit.com
6           aol.com
7         aoltw.net
8           aol.com
9           aol.com

Total de TLDs únicos: 106

TLDs más frecuentes:
domain_tld
aoltw.net         15
aol.com           11
google.com        10
stayonline.net     6
microsoft.com      6
home               4
mozilla.com        4
comcast.net        4
apple.com          4
me.com             3
Name: count, dtype: int64
